In [1]:
!pip install catboost[gpu]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tqdm import tqdm
import optuna
from optuna import Trial
from optuna.logging import set_verbosity, INFO
import catboost as cb
import gc

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [4]:
# id 열 삭제
train = train.drop(['id'], axis = 1)

In [5]:
# gender, vehicle_age, vehicle_damage 데이터 타입 변경 -> 라벨인코더로 통일
label_encoder = LabelEncoder()
cat=['Gender', 'Vehicle_Age', 'Vehicle_Damage']
for column in cat:
    train[column] = label_encoder.fit_transform(train[column])
    
for column in cat:
    test[column] = label_encoder.fit_transform(test[column])

In [6]:
# min_max_scaler
numerical = ['Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
scaler = MinMaxScaler()
train[numerical] = scaler.fit_transform(train[numerical])
train.head()

test[numerical] = scaler.fit_transform(test[numerical])

In [7]:
target = "Response"

In [8]:
X = train.drop(target, axis=1)
y = train[target]

In [9]:
# Stratified K-Fold 설정
n_splits = 2
skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [10]:
def objective(trial):
    params = {
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'task_type': 'GPU',
        'devices': '0',  
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 100.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'verbose': 0
    }

    skfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    oof_preds = []
    oof_aucs = []

    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
        
        train_pool = cb.Pool(data=X_train, label=y_train)
        eval_pool = cb.Pool(data=X_test, label=y_test)
        
        model = cb.CatBoostClassifier(**params)
        
        model.fit(train_pool, eval_set=eval_pool, early_stopping_rounds=50, verbose=0)
        
        y_pred = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred)
        
        oof_preds.append(y_pred)
        oof_aucs.append(roc_auc)
    
    return np.mean(oof_aucs)


In [11]:
# 진행 상황을 출력하는 콜백 함수 정의
def logging_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")
    
# Optuna 스터디 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, callbacks=[logging_callback])

# 최적의 결과 출력
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-07-24 04:12:10,708] A new study created in memory with name: no-name-1b93fde4-156f-4c9b-9472-8a56e8d7e1ed
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-24 04:14:13,876] Trial 0 finished with value: 0.8732562867923626 and parameters: {'learning_rate': 0.2677681984456135, 'depth': 8, 'l2_leaf_reg': 1.3436801823885703e-07, 'random_strength': 6.23308974404795, 'bagging_temperature': 1.142432250976999, 'border_count': 101}. Best is trial 0 with value: 0.8732562867923626.


Trial 0 finished with value: 0.8732562867923626 and parameters: {'learning_rate': 0.2677681984456135, 'depth': 8, 'l2_leaf_reg': 1.3436801823885703e-07, 'random_strength': 6.23308974404795, 'bagging_temperature': 1.142432250976999, 'border_count': 101}


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-24 04:17:01,569] Trial 1 finished with value: 0.8648641349682555 and parameters: {'learning_rate': 0.03855701232728146, 'depth': 4, 'l2_leaf_reg': 3.340705844220733e-05, 'random_strength': 2.188388189823275, 'bagging_temperature': 5.523954887332133, 'border_count': 123}. Best is trial 0 with value: 0.8732562867923626.


Trial 1 finished with value: 0.8648641349682555 and parameters: {'learning_rate': 0.03855701232728146, 'depth': 4, 'l2_leaf_reg': 3.340705844220733e-05, 'random_strength': 2.188388189823275, 'bagging_temperature': 5.523954887332133, 'border_count': 123}


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-24 04:19:17,241] Trial 2 finished with value: 0.8769983375183559 and parameters: {'learning_rate': 0.22699365018441214, 'depth': 10, 'l2_leaf_reg': 0.0017546732202033967, 'random_strength': 8.978746541278188, 'bagging_temperature': 1.3378771565305747, 'border_count': 192}. Best is trial 2 with value: 0.8769983375183559.


...

Trial 47 finished with value: 0.8775842122897004 and parameters: {'learning_rate': 0.14814593862005043, 'depth': 9, 'l2_leaf_reg': 0.4105577535462637, 'random_strength': 3.1654751328538877, 'bagging_temperature': 1.8698819976602319, 'border_count': 197}


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-24 07:03:21,146] Trial 48 finished with value: 0.8733444627233584 and parameters: {'learning_rate': 0.13024281516149386, 'depth': 9, 'l2_leaf_reg': 9.41610060986508e-06, 'random_strength': 9.628204721570178, 'bagging_temperature': 2.4854806944829795, 'border_count': 117}. Best is trial 10 with value: 0.8798317844001096.


Trial 48 finished with value: 0.8733444627233584 and parameters: {'learning_rate': 0.13024281516149386, 'depth': 9, 'l2_leaf_reg': 9.41610060986508e-06, 'random_strength': 9.628204721570178, 'bagging_temperature': 2.4854806944829795, 'border_count': 117}


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-24 07:08:16,244] Trial 49 finished with value: 0.8791858780873909 and parameters: {'learning_rate': 0.10858516727435769, 'depth': 10, 'l2_leaf_reg': 5.98897811301219, 'random_strength': 3.8671935750434185, 'bagging_temperature': 1.1887647841727615, 'border_count': 245}. Best is trial 10 with value: 0.8798317844001096.


Trial 49 finished with value: 0.8791858780873909 and parameters: {'learning_rate': 0.10858516727435769, 'depth': 10, 'l2_leaf_reg': 5.98897811301219, 'random_strength': 3.8671935750434185, 'bagging_temperature': 1.1887647841727615, 'border_count': 245}
Best trial:
  Value:  0.8798317844001096
  Params: 
    learning_rate: 0.19574552985670457
    depth: 6
    l2_leaf_reg: 0.8703543329753842
    random_strength: 4.956084588870899
    bagging_temperature: 0.04330122784989854
    border_count: 248


In [12]:
# 최적의 파라미터로 최종 모델 학습 및 평가
best_params = trial.params
model = cb.CatBoostClassifier(**best_params, task_type='GPU', devices='0', iterations=1000, verbose=0)

oof_preds = []
oof_aucs = []

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    train_pool = cb.Pool(data=X_train, label=y_train)
    eval_pool = cb.Pool(data=X_test, label=y_test)
    
    model.fit(train_pool, eval_set=eval_pool, early_stopping_rounds=50, verbose=0)
    
    y_pred = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred)
    
    oof_preds.append(y_pred)
    oof_aucs.append(roc_auc)
    
    print(f"\n---- Fold {fold}: ROC-AUC Score: {roc_auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    gc.collect()
    
auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} ± {auc_std:.6f}\n")


---- Fold 0: ROC-AUC Score: 0.879823


---- Fold 1: ROC-AUC Score: 0.879746


---- Fold 2: ROC-AUC Score: 0.879940


---- Fold 3: ROC-AUC Score: 0.879751


---- Fold 4: ROC-AUC Score: 0.880494


---> ROC-AUC Score: 0.879951 ± 0.000281



In [13]:
# Test set을 활용한 최종 예측
model.fit(X, y)
test_pred = model.predict_proba(test)[:, 1]

In [14]:
# 결과를 파일로 저장
submission = pd.DataFrame({'Id': test.index, 'Response': test_pred})
submission.to_csv('submission.csv', index=False)

## 전처리 없는 버전

In [16]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report
import catboost as cb

In [17]:
train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv", index_col = "id", engine="pyarrow")
test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv", index_col = "id", engine="pyarrow")

In [18]:
train["Region_Code"] = train["Region_Code"].astype(np.int8)
test["Region_Code"] = test["Region_Code"].astype(np.int8)

train["Policy_Sales_Channel"] = train["Policy_Sales_Channel"].astype(np.int16)
test["Policy_Sales_Channel"] = test["Policy_Sales_Channel"].astype(np.int16)

In [19]:
target="Response"

In [20]:
X = train.drop(target, axis=1)
y = train[target]

In [21]:
skfold = StratifiedKFold(2, shuffle=True, random_state=42)

In [23]:
def objective(trial):
    params = {
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'class_name' : [0, 1],
        'task_type': 'GPU', 
        'iterations': 3000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 100.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'max_leaves': trial.suggest_int('max_leaves', 1, 64),
        'fold_permutation_block': trial.suggest_int('fold_permutation_block', 1, 256),
        'verbose': False,
        'verbose': False,
        'allow_writing_files' : False,
        'random_seed' : 42
    }

    skfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    oof_preds = []
    oof_aucs = []

    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
        
        train_pool = cb.Pool(data=X_train, label=y_train)
        eval_pool = cb.Pool(data=X_test, label=y_test)
        
        model = cb.CatBoostClassifier(**params)
        
        model.fit(train_pool, eval_set=eval_pool, early_stopping_rounds=50, verbose=0)
        
        y_pred = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred)
        
        oof_preds.append(y_pred)
        oof_aucs.append(roc_auc)
    
    return np.mean(oof_aucs)

In [24]:
# 진행 상황을 출력하는 콜백 함수 정의
def logging_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")
    
# Optuna 스터디 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, callbacks=[logging_callback])

# 최적의 결과 출력
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-07-24 07:20:35,348] A new study created in memory with name: no-name-c797c7c0-df61-4916-b6bf-9f45e6884865
[W 2024-07-24 07:20:39,428] Trial 0 failed with parameters: {'learning_rate': 0.19067707754980925, 'depth': 4, 'l2_leaf_reg': 5.902351172975457e-05, 'random_strength': 8.961194266140799, 'bagging_temperature': 6.178061556213699, 'border_count': 1, 'max_leaves': 3, 'fold_permutation_block': 116} because of the following error: CatBoostError('Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Male": Cannot convert \'b\'Male\'\' to float').
Traceback (most recent call last):
  File "_catboost.pyx", line 2383, in _catboost.get_float_feature
  File "_catboost.pyx", line 1188, in _catboost._FloatOrNan
  File "_catboost.pyx", line 983, in _catboost._FloatOrNanFromString
TypeError: Cannot convert 'b'Male'' to float

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/opt

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Male": Cannot convert 'b'Male'' to float

In [ ]:
# Test set을 활용한 최종 예측
model.fit(X, y)
test_pred = model.predict_proba(test)[:, 1]

In [ ]:
# 결과를 파일로 저장
submission = pd.DataFrame({'Id': test.index, 'Response': test_pred})
submission.to_csv('submission2.csv', index=False)